In [1]:
# !pip install --quiet  datasets #to access squad dataset
# !pip install --quiet pyarrow   #to deal with parquet files for saving dataset if required
# !pip install --quiet  tqdm     #for progress bars
# !pip install --quiet transformers # for t5 model
# !pip install --quiet tokenizers  #tokenizers from HuggingFace
# !pip install --quiet sentencepiece #subword tokenizer used by T5
# !pip install --quiet pytorch-lightning # pytorch wrapper 
# !pip install --quiet torchtext # text utilities

# Fetching Datasets

In [2]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import DatasetDict, Dataset, load_from_disk
# from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy
import numpy as np
from collections import defaultdict
import ipdb

In [3]:
device  = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# path_to_source = f"/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_full_5_10_10000_clone_latex_compare/10Neg10000unk/twofoldwithunk"
# path_to_csv = f"/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/twofoldwithunk"
path_to_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/zero-shot/pwc_pdf_5_10_800/twofoldwithunk"

fold1 = "fold1"
train_f1_pd = pd.read_csv(f"{path_to_csv}/{fold1}/train.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])
dev_f1_pd = pd.read_csv(f"{path_to_csv}/{fold1}/dev.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])

fold2 = "fold2"
train_f2_pd = pd.read_csv(f"{path_to_csv}/{fold2}/train.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])
dev_f2_pd = pd.read_csv(f"{path_to_csv}/{fold2}/dev.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])

In [6]:
pwd

'/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/notebooks'

In [7]:
# # no_leaderboard_pd = pd.read_csv(f"/nfs/home/kabenamualus/Research/T5-Leaderboard-QA/data_proccess/arxiv_no_leaderboard_links_pdf_short/DocTAET_full.tsv", 
# #                     sep="\t", names=["title", "Context"])

# no_leaderboard_pd = pd.read_csv(f"/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_no_leaderboard_links_pdf_short/DocTAET_150.tsv", 
#                     sep="\t", names=["title", "Context"])

# no_leaderboard_pd.describe()

In [8]:
resultsAnnotation_pd = pd.read_csv(f"/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/annotations_dec092023/resultsAnnotation.tsv",
                                   sep="\t", names=["Title", "TDMSs"])
resultsAnnotation_pd = resultsAnnotation_pd.fillna("NAN")
resultsAnnotation_pd

,Title,TDMSs
0,1704.03549v4.pdf,Optical Character Recognition (OCR)#FSNS - Tes...
1,1712.05404.pdf,Optical Character Recognition (OCR)#FSNS - Tes...
2,1702.03970v1.pdf,Optical Character Recognition (OCR)#FSNS - Tes...
3,1802.05415v2.pdf,Optical Character Recognition (OCR)#im2latex-1...
4,2308.15996v1.pdf,Optical Character Recognition (OCR)#Benchmarki...
...,...,...
12728,2112.07910v2.pdf,Open Vocabulary Semantic Segmentation#COCO-Stu...
12729,1709.08011v3.pdf,Japanese Word Segmentation#BCCWJ#F1-score (Wor...
12730,2203.04616v2.pdf,Binary Condescension Detection#DPM#F1-score#63...
12731,2208.01312v1.pdf,Binary Condescension Detection#DPM#F1-score#61...


In [9]:
"""
This will take care of papers with more than one learderboard 
"""
records = resultsAnnotation_pd.to_dict("records")

title_to_tdms_dict = defaultdict(
    lambda : 
        list()
    )

for i, row in tqdm(enumerate(records), total = len(records)):
    if row['TDMSs'] == 'NAN':
        continue

    for tdms in row['TDMSs'].split("$"):
        if len(tdms.split("#")) != 4:
            # ipdb.set_trace()
            continue 
        t, d, m, s = tdms.split("#")
        title_to_tdms_dict[row['Title']].append(
            {
                "LEADERBOARD": {
                    "Task": t,
                    "Dataset": d,
                    "Metric": m,
                    "Score": s,
                }
            }            
        )

100%|██████████| 12733/12733 [00:00<00:00, 32030.59it/s]


In [10]:
len(title_to_tdms_dict)

12718

In [11]:
# No need for negative instances, but will still have 'duplicate' for paper with more than one leaderboard
train_f1_pd = train_f1_pd[train_f1_pd.label==True]
print("train_f1_pd")
display(train_f1_pd.describe())

dev_f1_pd = dev_f1_pd[dev_f1_pd.label==True]
print("dev_f1_pd")
display(dev_f1_pd.describe())

train_f2_pd = train_f2_pd[train_f2_pd.label==True]
print("train_f2_pd")
display(train_f2_pd.describe())

dev_f2_pd = dev_f2_pd[dev_f2_pd.label==True]
print("dev_f2_pd")
display(dev_f2_pd.describe())

train_f1_pd


,label,title,TDM,Context
count,25859,25859,25859,25859
unique,1,5636,3472,5634
top,True,2206.03192v4.pdf,unknown,Generalized Data Distribution Iteration To obt...
freq,25859,62,588,62


dev_f1_pd


,label,title,TDM,Context
count,11085,11085,11085,11085
unique,1,2415,2898,2415
top,True,1802.01561v3.pdf,unknown,IMPALA: Scalable Distributed Deep-RL with Impo...
freq,11085,61,212,61


train_f2_pd


,label,title,TDM,Context
count,25628,25628,25628,25628
unique,1,5652,3436,5650
top,True,1802.01561v3.pdf,unknown,IMPALA: Scalable Distributed Deep-RL with Impo...
freq,25628,61,507,61


dev_f2_pd


,label,title,TDM,Context
count,11385,11385,11385,11385
unique,1,2416,2913,2415
top,True,2206.03192v4.pdf,unknown,Generalized Data Distribution Iteration To obt...
freq,11385,62,293,62


In [12]:
# len(train_pd.title.unique())
records_train_f1 = train_f1_pd.to_dict("records")
records_dev_f1 = dev_f1_pd.to_dict("records")
records_train_f2 = train_f2_pd.to_dict("records")
records_dev_f2 = dev_f2_pd.to_dict("records")

In [13]:
title_id = records_train_f1[0]["title"].split(".pdf")[0]
title_id

'2006.10721v2'

In [14]:
# title_to_tdms_dict = defaultdict(lambda : defaultdict(lambda : str("| ")))
title_to_content = {
    "train_f1":{},
    "dev_f1":{},
    "train_f2":{},
    "dev_f2":{},
    }

# arxiv_leaderboard_full_txt = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_leaderboard_full_txt"
arxiv_leaderboard_full_txt = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_txt_summarised_dec092023"

missed = 0
for i, row in tqdm(enumerate(records_train_f1), total = len(records_train_f1)):
    title_id = row['title'].split(".pdf")[0]
    if row['title'] in title_to_content["train_f1"]:
        continue 
    else:
        try:
            with open(f'{arxiv_leaderboard_full_txt}/{title_id}_summarised.txt', 'r') as file:
                # Read the file
                data = file.read()
                
        except :
            # print(f"Error on file {row['title']}")
            data = "" 
            missed += 1
            continue

        if len(data.split()) < 10:
            continue
        else:
            title_to_content["train_f1"][row['title']] = data
        # title_to_content["train_f1"][row['title']] = row['Context'] if len(data.split()) < 100 else data
        # title_to_content["train_f1"][row['title']] = row['Context']

print(f"train_f1 missed long context: {missed}/{len(records_train_f1)}\n")

missed = 0
for i, row in tqdm(enumerate(records_dev_f1), total = len(records_dev_f1)):
    title_id = row['title'].split(".pdf")[0]
    if row['title'] in title_to_content["dev_f1"]:
        continue 
    else:
        try:
            with open(f'{arxiv_leaderboard_full_txt}/{title_id}_summarised.txt', 'r') as file:
                # Read the file
                data = file.read()
                
        except :
            # print(f"Error on file {row['title']}")
            data = ""
            missed += 1
            # continue 
        
        title_to_content["dev_f1"][row['title']] = row['Context'] if len(data.split()) < 100 else data

print(f"dev_f1 missed long context: {missed}/{len(records_dev_f1)}\n")       
        
missed = 0        
for i, row in tqdm(enumerate(records_train_f2), total = len(records_train_f2)):
    title_id = row['title'].split(".pdf")[0]
    if row['title'] in title_to_content["train_f2"]:
        continue 
    else:
        try:
            with open(f'{arxiv_leaderboard_full_txt}/{title_id}_summarised.txt', 'r') as file:
                # Read the file
                data = file.read()
                
        except :
            # print(f"Error on file {row['title']}")
            data = ""
            missed += 1
        
        title_to_content["train_f2"][row['title']] = row['Context'] if len(data.split()) < 100 else data
           
print(f"train_f2 missed long context: {missed}/{len(records_train_f2)}\n")       

missed = 0
for i, row in tqdm(enumerate(records_dev_f2), total = len(records_dev_f2)):
    title_id = row['title'].split(".pdf")[0]
    if row['title'] in title_to_content["dev_f2"]:
        continue 
    else:
        try:
            with open(f'{arxiv_leaderboard_full_txt}/{title_id}_summarised.txt', 'r') as file:
                # Read the file
                data = file.read()
                
        except :
            # print(f"Error on file {row['title']}")
            data = ""
            missed += 1
            # continue
            
        title_to_content["dev_f2"][row['title']] = row['Context'] if len(data.split()) < 100 else data
        # title_to_content["dev_f2"][row['title']] = row['Context']

print(f"train_f2 missed long context: {missed}/{len(records_dev_f2)}")       

100%|██████████| 25859/25859 [00:54<00:00, 476.23it/s] 


train_f1 missed long context: 8646/25859



100%|██████████| 11085/11085 [00:24<00:00, 461.04it/s]


dev_f1 missed long context: 769/11085



100%|██████████| 25628/25628 [00:01<00:00, 13452.25it/s]


train_f2 missed long context: 1819/25628



100%|██████████| 11385/11385 [00:00<00:00, 26205.22it/s]

train_f2 missed long context: 771/11385


In [17]:
# no_leaderboard_pourcentage_train_f1 = int(len(train_f1_pd.title.unique())*50/100)
# no_leaderboard_pourcentage_dev_f1 = int(len(dev_f1_pd.title.unique())*50/100)
# no_leaderboard_pourcentage_train_f2 = int(len(train_f2_pd.title.unique())*50/100)
# no_leaderboard_pourcentage_dev_f2 = int(len(dev_f2_pd.title.unique())*50/100)

# print(f"no_leaderboard_pourcentage_train_f1: {no_leaderboard_pourcentage_train_f1}")
# print(f"no_leaderboard_pourcentage_dev_f1: {no_leaderboard_pourcentage_dev_f1}")
# print(f"no_leaderboard_pourcentage_train_f2: {no_leaderboard_pourcentage_train_f2}")
# print(f"no_leaderboard_pourcentage_dev_f2: {no_leaderboard_pourcentage_dev_f2}")

### Add additional Training data

In [20]:
# no_leaderboard_pourcentage = int(len(train_pd.title.unique())*50/100)
# no_leaderboard_pourcentage

In [14]:
records = no_leaderboard_pd.to_dict("records")

# For train only F1
no_lead_papers_train_f1 = []

already_seen = no_lead_papers_train_f1
no_lead_papers_train_f1 = []
i = 0

# arxiv_no_leaderboard_full_txt = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_no_leaderboard_full_txt"
arxiv_no_leaderboard_full_txt = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_no_leaderboard_txt_25_000_summarised_dec092023"

missed = 0
for _, row in tqdm(enumerate(records), total = len(records)):
    title_id = row['title'].split(".pdf")[0]
    if row['title'] in already_seen:
        continue 
        
    if i >= no_leaderboard_pourcentage_train_f1:
        break 
    
    try:
        with open(f'{arxiv_no_leaderboard_full_txt}/{title_id}.txt', 'r') as file:
            # Read the file
            data = file.read()
            
    except :
        # print(f"Error on file {row['title']}")
        data = ""
        missed += 1
                
    title_to_content["train_f1"][row['title']] = row['Context'] if len(data.split()) < 100 else data
    no_lead_papers_train_f1.append(row['title'])
    i += 1
print(f"train_f1 missed long context: {missed}/{i}\n")       
    
no_lead_papers_dev_f1 = no_lead_papers_train_f1
already_seen = no_lead_papers_dev_f1
no_lead_papers_dev_f1 = []
i = 0
missed = 0
for _, row in tqdm(enumerate(records), total = len(records)):
    title_id = row['title'].split(".pdf")[0]
    if row['title'] in already_seen:
        continue 
        
    if i >= no_leaderboard_pourcentage_dev_f1:
        break 
     
    try:
        with open(f'{arxiv_no_leaderboard_full_txt}/{title_id}.txt', 'r') as file:
            # Read the file
            data = file.read()
            
    except :
        # print(f"Error on file {row['title']}")
        data = ""
        missed += 1
           
    title_to_content["dev_f1"][row['title']] = row['Context'] if len(data.split()) < 100 else data
    no_lead_papers_dev_f1.append(row['title'])  
    i += 1
print(f"dev_f1 missed long context: {missed}/{i}\n")       

    
# For train only F2
no_lead_papers_train_f2 = []

already_seen = no_lead_papers_train_f2
no_lead_papers_train_f2 = []
j = 0
missed = 0
for _, row in tqdm(enumerate(records), total = len(records)):
    title_id = row['title'].split(".pdf")[0]
    if row['title'] in already_seen:
        continue 
        
    if j >= no_leaderboard_pourcentage_train_f2:
        break 
    
    try:
        with open(f'{arxiv_no_leaderboard_full_txt}/{title_id}.txt', 'r') as file:
            # Read the file
            data = file.read()
            
    except :
        # print(f"Error on file {row['title']}")
        data = ""
        missed += 1
        
    title_to_content["train_f2"][row['title']] = row['Context'] if len(data.split()) < 100 else data
    no_lead_papers_train_f2.append(row['title'])
    j += 1
print(f"train_f2 missed long context: {missed}/{j}\n")  
    
no_lead_papers_dev_f2 = no_lead_papers_train_f2
already_seen = no_lead_papers_dev_f2
no_lead_papers_dev_f2 = []
j = 0
missed = 0
for _, row in tqdm(enumerate(records), total = len(records)):
    title_id = row['title'].split(".pdf")[0]
    if row['title'] in already_seen:
        continue 
        
    if j >= no_leaderboard_pourcentage_dev_f2:
        break 
        
    try:
        with open(f'{arxiv_no_leaderboard_full_txt}/{title_id}.txt', 'r') as file:
            # Read the file
            data = file.read()
            
    except :
        # print(f"Error on file {row['title']}")
        data = ""
        missed += 1
        
    title_to_content["dev_f2"][row['title']] = row['Context'] if len(data.split()) < 100 else data
    no_lead_papers_dev_f2.append(row['title'])
    j += 1
print(f"train_f2 missed long context: {missed}/{j}\n")  

  7%|▋         | 314/4369 [00:00<00:01, 3136.98it/s]

 43%|████▎     | 1876/4369 [00:00<00:00, 3109.13it/s]


train_f1 missed long context: 1876/1876



 61%|██████▏   | 2680/4369 [00:00<00:00, 7505.22it/s] 


dev_f1 missed long context: 804/804



 43%|████▎     | 1876/4369 [00:00<00:00, 50867.33it/s]


train_f2 missed long context: 1876/1876



 61%|██████▏   | 2680/4369 [00:00<00:00, 39337.52it/s]

train_f2 missed long context: 804/804



In [16]:
# train_f1_pd["Lenght context"] = train_f1_pd.Context.apply(lambda x: len(x.split()))
# dev_f1_pd["Lenght context"] = dev_f1_pd.Context.apply(lambda x: len(x.split()))
# train_f2_pd["Lenght context"] = train_f2_pd.Context.apply(lambda x: len(x.split()))
# dev_f2_pd["Lenght context"] = dev_f2_pd.Context.apply(lambda x: len(x.split()))

In [17]:
# train_pd[train_pd["Lenght context"] < 400]

In [18]:
# train_pd = train_pd[train_pd["Lenght context"] < 400]

In [19]:
# print("train_f1_pd describe: ")
# display(train_f1_pd.describe())
# print("dev_f1_pd describe: ")
# display(dev_f1_pd.describe())

# print("train_f2_pd describe: ")
# display(train_f2_pd.describe())
# print("dev_f2_pd describe: ")
# display(dev_f2_pd.describe())

In [15]:
df_train_f1 = pd.DataFrame(columns = ["Title", "TDMSs", "Context"])
for i, title in tqdm(enumerate(title_to_content["train_f1"].keys()), total = len(title_to_content["train_f1"].keys())):
    
    if (len(title_to_content["train_f1"][title]) < 10):
        continue 
    
    if (title not in no_lead_papers_train_f1) :
        if (title_to_tdms_dict[title] == []):
             continue

    df_train_f1 = pd.concat([df_train_f1, pd.DataFrame.from_records(
        [
            {
                'Title' : title, 
                'TDMSs' : title_to_tdms_dict[title] if title in title_to_tdms_dict.keys() else "unanswerable",
                'Context' : title_to_content["train_f1"][title],
                'Lenght Context': len(title_to_content["train_f1"][title].split()),
                'Lenght TDMSs': len(str(title_to_tdms_dict[title] if title in title_to_tdms_dict.keys() else "unanswerable").split())
            }
        ])], ignore_index = True)
print("df_train_f1 describe: ")
display(df_train_f1.describe())  

df_dev_f1 = pd.DataFrame(columns = ["Title", "TDMSs", "Context"])  
for i, title in tqdm(enumerate(title_to_content["dev_f1"].keys()), total = len(title_to_content["dev_f1"].keys())):
    
    if (len(title_to_content["dev_f1"][title]) < 10):
        continue 
    
    if (title not in no_lead_papers_dev_f1) :
        if (title_to_tdms_dict[title] == []):
             continue

    df_dev_f1 = pd.concat([df_dev_f1, pd.DataFrame.from_records(
        [
            {
                'Title' : title, 
                'TDMSs' : title_to_tdms_dict[title] if title in title_to_tdms_dict.keys() else "unanswerable",
                'Context' : title_to_content["dev_f1"][title],
                'Lenght Context': len(title_to_content["dev_f1"][title].split()),
                'Lenght TDMSs': len(str(title_to_tdms_dict[title] if title in title_to_tdms_dict.keys() else "unanswerable").split())
            }
        ])], ignore_index = True)
print("df_dev_f1 describe: ")
display(df_dev_f1.describe())  

df_train_f2 = pd.DataFrame(columns = ["Title", "TDMSs", "Context"])
for i, title in tqdm(enumerate(title_to_content["train_f2"].keys()), total = len(title_to_content["train_f2"].keys())):
    
    if (len(title_to_content["train_f2"][title]) < 10):
        continue 
    
    if (title not in no_lead_papers_train_f2) :
        if (title_to_tdms_dict[title] == []):
             continue

    df_train_f2 = pd.concat([df_train_f2, pd.DataFrame.from_records(
        [
            {
                'Title' : title, 
                'TDMSs' : title_to_tdms_dict[title] if title in title_to_tdms_dict.keys() else "unanswerable",
                'Context' : title_to_content["train_f2"][title],
                'Lenght Context': len(title_to_content["train_f2"][title].split()),
                'Lenght TDMSs': len(str(title_to_tdms_dict[title] if title in title_to_tdms_dict.keys() else "unanswerable").split())
            }
        ])], ignore_index = True)
print("df_train_f2 describe: ")
display(df_train_f2.describe())  
 
df_dev_f2 = pd.DataFrame(columns = ["Title", "TDMSs", "Context"])  
for i, title in tqdm(enumerate(title_to_content["dev_f2"].keys()), total = len(title_to_content["dev_f2"].keys())):
    
    if (len(title_to_content["dev_f2"][title]) < 10):
        continue 
    
    if (title not in no_lead_papers_dev_f2) :
        if (title_to_tdms_dict[title] == []):
             continue

    df_dev_f2 = pd.concat([df_dev_f2, pd.DataFrame.from_records(
        [
            {
                'Title' : title, 
                'TDMSs' : title_to_tdms_dict[title] if title in title_to_tdms_dict.keys() else "unanswerable",
                'Context' : title_to_content["dev_f2"][title],
                'Lenght Context': len(title_to_content["dev_f2"][title].split()),
                'Lenght TDMSs': len(str(title_to_tdms_dict[title] if title in title_to_tdms_dict.keys() else "unanswerable").split())
            }
        ])], ignore_index = True)
print("df_dev_f2 describe: ")
display(df_dev_f2.describe())  

  5%|▍         | 256/5629 [00:00<00:04, 1307.73it/s]

100%|██████████| 5629/5629 [00:04<00:00, 1345.66it/s]


df_train_f1 describe: 


,Lenght Context,Lenght TDMSs
count,5512.000000,5512.000000
mean,318.346154,46.065312
std,167.995792,102.111303
min,3.000000,1.000000
25%,194.000000,1.000000
50%,323.000000,20.000000
75%,419.000000,52.000000
max,2510.000000,2963.000000


100%|██████████| 2412/2412 [00:01<00:00, 1410.98it/s]

...

df_train_f2 describe: 


,Lenght Context,Lenght TDMSs
count,5513.000000,5513.000000
mean,319.830219,46.404498
std,169.799786,100.723495
min,3.000000,1.000000
25%,194.000000,1.000000
50%,324.000000,20.000000
75%,423.000000,52.000000
max,2510.000000,2963.000000


100%|██████████| 2412/2412 [00:01<00:00, 1431.06it/s]

df_dev_f2 describe: 


,Lenght Context,Lenght TDMSs
count,2352.000000,2352.000000
mean,318.417942,44.800170
std,155.981723,90.045289
min,4.000000,1.000000
25%,197.000000,1.000000
50%,326.000000,20.000000
75%,417.000000,53.000000
max,1481.000000,1829.000000


In [16]:
template = ['Please answer a question about this article. If the question is unanswerable, say \"unanswerable\"',
'Read this and answer the question. If the question is unanswerable, say \"unanswerable\".',
'If the question is unanswerable, say \"unanswerable\"',
'Try to answer this question if possible (otherwise reply \"unanswerable\"',
'If it is possible to answer this question, answer it for me (else, reply \"unanswerable\"',
'Answer this question, if possible (if impossible, reply \"unanswerable\"',
'Read this: What is the answer? (If it cannot be answered, return \"unanswerable\"',
'Read this: Now answer this question, if there is an answer (If it cannot be answered, return \"unanswerable\"',
'Answer based on context:',
'Answer this question based on the article:',
# ("{context}\n\n{question}", "{answer}"),
'Answer this question:',
'Read this article and answer this question',
'Based on the above article, answer a question.',
# 'Context: {context}\n\nQuestion: {question}\n\nAnswer:", "{answer}"),
]

template

['Please answer a question about this article. If the question is unanswerable, say "unanswerable"',
 'Read this and answer the question. If the question is unanswerable, say "unanswerable".',
 'If the question is unanswerable, say "unanswerable"',
 'Try to answer this question if possible (otherwise reply "unanswerable"',
 'If it is possible to answer this question, answer it for me (else, reply "unanswerable"',
 'Answer this question, if possible (if impossible, reply "unanswerable"',
 'Read this: What is the answer? (If it cannot be answered, return "unanswerable"',
 'Read this: Now answer this question, if there is an answer (If it cannot be answered, return "unanswerable"',
 'Answer based on context:',
 'Answer this question based on the article:',
 'Answer this question:',
 'Read this article and answer this question',
 'Based on the above article, answer a question.']

In [22]:
# df.head(2)

In [17]:
def create_pandas_dataset_from_pandas(df,
                                      squad_1 = False,
                                      squad_2 = False,
                                      squad_3 = False,
                                      squad_4 = False,
                                      squad_5 = False,
                                      squad_6 = False,
                                      squad_7 = False,
                                      squad_8 = False,
                                      drop_1 = False,
                                      drop_2 = False,
                                      drop_3 = False,
                                      drop_4 = False,
                                      drop_5 = False,
                                      drop_6 = False,
                                      drop_7 = False
                         ):

  ''' Create a Pandas Dataframe from pandas.
  Params:
        answer_threshold: Only consider those Question Answer pairs where the Answer is short.
  '''
  count_index = 0
  result_df  = pd.DataFrame(columns = ['prompt', 'answer'])   
  # q_type_1 = "Which Tasks are addressed in this article"
  # q_type_2 = "Which Datasets are addressed in this article"
  # q_type_3 = "Which Metrics are addressed in this article"
  # q_type_4 = "Which Tasks, Datasets, Metrics are addressed in this article"
  # q_type_5 = "Which Tasks, Datasets, Metrics and Scores are addressed in this article" 
  
  # q_types = [
  #   {"q": "Which Tasks are addressed in this article", "a_key": "Tasks"}, 
  #   {"q": "Which Datasets are addressed in this article", "a_key": "Datasets"}, 
  #   {"q": "Which Metrics are addressed in this article", "a_key": "Metrics"},
  #   {"q": "Which Tasks, Datasets, Metrics are addressed in this article", "a_key": "TDMs"},
  #   {"q": "Which Tasks, Datasets, Metrics and Scores are addressed in this article", "a_key": "TDMSs"}
  #   ]
  
  q_types = [
    # {"q": "What are the values for the following properties to construct a Leaderboard for the model introduced in this article: task, dataset, and metric?", "a_key": "TDMSs"},
    {"q": "What are the values for the following properties to construct a Leaderboard for the model introduced in this article: task, dataset, metric, and score?", "a_key": "TDMSs"},
    ]
  
  records = df.to_dict("records")
  # db_dict = defaultdict(lambda : list())
  for i, row in tqdm(enumerate(records), total = len(records)):        
      for q_type in q_types:
        
        # Squad_v2 
        if squad_1:
          result_df.loc[count_index] = [f'{row["Context"]}\n\nPlease answer a question about this article. If the question is unanswerable, say \"unanswerable\". {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if squad_2:
          result_df.loc[count_index] = [f'Read this and answer the question. If the question is unanswerable, say \"unanswerable\".\n\n{row["Context"]}\n\n{q_type["q"]}'
  ] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if squad_3:
          result_df.loc[count_index] = [f'{row["Context"]}\n{q_type["q"]} (If the question is unanswerable, say \"unanswerable\"'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if squad_4:
          result_df.loc[count_index] = [f'{row["Context"]}\nTry to answer this question if possible (otherwise reply \"unanswerable\"): {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if squad_5:
          result_df.loc[count_index] = [f'{row["Context"]}\nIf it is possible to answer this question, answer it for me (else, reply \"unanswerable\"): {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if squad_6:
          result_df.loc[count_index] = [f'{row["Context"]}\n\nAnswer this question, if possible (if impossible, reply \"unanswerable\"): {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if squad_7:
          result_df.loc[count_index] = [f'Read this: {row["Context"]}\n\n{q_type["q"]}\nWhat is the answer? (If it cannot be answered, return \"unanswerable\")'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if squad_8:
          result_df.loc[count_index] = [f'Read this: {row["Context"]}\nNow answer this question, if there is an answer (If it cannot be answered, return \"unanswerable\"): {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        
        # Drop
        if drop_1:
          result_df.loc[count_index] = [f'Answer based on context:\n\n{row["Context"]}\n\n{q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if drop_2:
          result_df.loc[count_index] = [f'{row["Context"]}\n\nAnswer this question based on the article: {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if drop_3:
          result_df.loc[count_index] = [f'{row["Context"]}\n\n{q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if drop_4:
          result_df.loc[count_index] = [f'{row["Context"]}\nAnswer this question: {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if drop_5:
          result_df.loc[count_index] = [f'Read this article and answer this question {row["Context"]}\n{q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if drop_6:
          result_df.loc[count_index] = [f'{row["Context"]}\n\nBased on the above article, answer a question. {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if drop_7:
          result_df.loc[count_index] = [f'Context: {row["Context"]}\n\nQuestion: {q_type["q"]}\n\nAnswer:'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
         
  return result_df

In [18]:
df_train_f1_all_templates = create_pandas_dataset_from_pandas(df_train_f1,
                                                                squad_1 = True,
                                                                squad_2 = True,
                                                                squad_3 = True,
                                                                squad_4 = True,
                                                                squad_5 = True,
                                                                squad_6 = True,
                                                                squad_7 = True,
                                                                squad_8 = True,
                                                                drop_1 = True,
                                                                drop_2 = True,
                                                                drop_3 = True,
                                                                drop_4 = True,
                                                                drop_5 = True,
                                                                drop_6 = True,
                                                                drop_7 = True
                                                              ) 
df_dev_f1_all_templates = create_pandas_dataset_from_pandas(df_dev_f1,
                                                            squad_1 = True,
                                                            squad_2 = True,
                                                            squad_3 = True,
                                                            squad_4 = True,
                                                            squad_5 = True,
                                                            squad_6 = True,
                                                            squad_7 = True,
                                                            squad_8 = True,
                                                            drop_1 = True,
                                                            drop_2 = True,
                                                            drop_3 = True,
                                                            drop_4 = True,
                                                            drop_5 = True,
                                                            drop_6 = True,
                                                            drop_7 = True
                                                            ) 
df_train_f2_all_templates = create_pandas_dataset_from_pandas(df_train_f2,
                                                            squad_1 = True,
                                                            squad_2 = True,
                                                            squad_3 = True,
                                                            squad_4 = True,
                                                            squad_5 = True,
                                                            squad_6 = True,
                                                            squad_7 = True,
                                                            squad_8 = True,
                                                            drop_1 = True,
                                                            drop_2 = True,
                                                            drop_3 = True,
                                                            drop_4 = True,
                                                            drop_5 = True,
                                                            drop_6 = True,
                                                            drop_7 = True
                                                              ) 
df_dev_f2_all_templates = create_pandas_dataset_from_pandas(df_dev_f2,
                                                            squad_1 = True,
                                                            squad_2 = True,
                                                            squad_3 = True,
                                                            squad_4 = True,
                                                            squad_5 = True,
                                                            squad_6 = True,
                                                            squad_7 = True,
                                                            squad_8 = True,
                                                            drop_1 = True,
                                                            drop_2 = True,
                                                            drop_3 = True,
                                                            drop_4 = True,
                                                            drop_5 = True,
                                                            drop_6 = True,
                                                            drop_7 = True 
                                                            ) 

print("df_train_f1_all_templates describe: ")
display(df_train_f1_all_templates.describe())
print("df_dev_f1_all_templates describe: ")
display(df_dev_f1_all_templates.describe())

print("df_train_f2_all_templates describe: ")
display(df_train_f2_all_templates.describe())
print("df_dev_f2_all_templates describe: ")
display(df_dev_f2_all_templates.describe())

  0%|          | 6/5512 [00:00<01:44, 52.81it/s]

100%|██████████| 2352/2352 [00:56<00:00, 41.97it/s]


df_train_f1_all_templates describe: 


,prompt,answer
count,82680,82680
unique,82650,3634
top,Read this and answer the question. If the ques...,unanswerable
freq,2,28080


df_dev_f1_all_templates describe: 


,prompt,answer
count,35295,35295
unique,35280,1549
top,PANDA: Adapting Pretrained Features for Anomal...,unanswerable
freq,2,12060


df_train_f2_all_templates describe: 


,prompt,answer
count,82695,82695
unique,82665,3637
top,IEEE TRANSACTIONS ON PATTERN ANALYSIS AND MACH...,unanswerable
freq,2,28080


df_dev_f2_all_templates describe: 


,prompt,answer
count,35280,35280
unique,35280,1548
top,Ocean: Object-aware Anchor-free Tracking Ancho...,unanswerable
freq,1,12060


In [19]:
df_train_f1_all_templates.describe()

,prompt,answer
count,82680,82680
unique,82650,3634
top,Read this and answer the question. If the ques...,unanswerable
freq,2,28080


In [20]:
df_train_f1_all_templates['answer'].apply(type).value_counts()


<class 'str'>    82680
Name: answer, dtype: int64

In [21]:
df_dev_f1_all_templates['answer'].apply(type).value_counts()

<class 'str'>    35295
Name: answer, dtype: int64

In [22]:
str(df_dev_f1_all_templates.at[5, 'answer'])

"[{'LEADERBOARD': {'Task': 'Semantic Segmentation', 'Dataset': 'Nighttime Driving', 'Metric': 'mIoU', 'Score': '36.1'}}]"

In [23]:
df_train_f1_all_templates.to_parquet('../data/df_train_tdms_long_summarized_f1_all_templates.parquet')
df_dev_f1_all_templates.to_parquet('../data/df_dev_tdms_long_summarized_f1_all_templates.parquet')
df_train_f2_all_templates.to_parquet('../data/df_train_tdms_long_summarized_f2_all_templates.parquet')
df_dev_f2_all_templates.to_parquet('../data/df_dev_tdms_long_summarized_f2_all_templates.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_all_templates.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_all_templates.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_all_templates.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_all_templates.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_ALL_TEMPLATE")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-322fd51c22294381/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-aedb7dfe9a5ce69b/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 82680
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 35295
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 82695
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 35280
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/82680 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/35295 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/82695 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/35280 [00:00<?, ? examples/s]

In [30]:
# df_train_f1_all_templates = pd.read_parquet('../data/df_train_f1_all_templates.parquet')
# df_dev_f1_all_templates = pd.read_parquet('../data/df_dev_f1_all_templates.parquet')
# df_train_f2_all_templates = pd.read_parquet('../data/df_train_f2_all_templates.parquet')
# df_dev_f2_all_templates = pd.read_parquet('../data/df_dev_f2_all_templates.parquet')

# Specific Template

In [24]:
df_train_f1_squad_1 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        squad_1 = True,
                                                        ) 
df_dev_f1_squad_1 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        squad_1 = True,
                                                        ) 
df_train_f2_squad_1 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        squad_1 = True,
                                                        ) 
df_dev_f2_squad_1 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        squad_1 = True,
                                                        ) 

print("df_train_f1_squad_1 describe: ")
display(df_train_f1_squad_1.describe())
print("df_dev_f1_squad_1 describe: ")
display(df_dev_f1_squad_1.describe())

print("df_train_f2_squad_1 describe: ")
display(df_train_f2_squad_1.describe())
print("df_dev_f2_squad_1 describe: ")
display(df_dev_f2_squad_1.describe())

df_train_f1_squad_1.to_parquet('../data/df_train_tdms_long_summarized_f1_squad_1.parquet')
df_dev_f1_squad_1.to_parquet('../data/df_dev_tdms_long_summarized_f1_squad_1.parquet')
df_train_f2_squad_1.to_parquet('../data/df_train_tdms_long_summarized_f2_squad_1.parquet')
df_dev_f2_squad_1.to_parquet('../data/df_dev_tdms_long_summarized_f2_squad_1.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_squad_1.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_squad_1.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_squad_1.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_squad_1.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_SQUAD_1")

  1%|▏         | 76/5512 [00:00<00:07, 752.67it/s]

100%|██████████| 2352/2352 [00:02<00:00, 845.10it/s]

df_train_f1_squad_1 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable
freq,2,1872


df_dev_f1_squad_1 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-0b72f258f6989d6c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [25]:
df_train_f1_squad_2 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        squad_2 = True,
                                                        ) 
df_dev_f1_squad_2 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        squad_2 = True,
                                                        ) 
df_train_f2_squad_2 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        squad_2 = True,
                                                        ) 
df_dev_f2_squad_2 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        squad_2 = True,
                                                        ) 

print("df_train_f1_squad_2 describe: ")
display(df_train_f1_squad_2.describe())
print("df_dev_f1_squad_2 describe: ")
display(df_dev_f1_squad_2.describe())

print("df_train_f2_squad_2 describe: ")
display(df_train_f2_squad_2.describe())
print("df_dev_f2_squad_2 describe: ")
display(df_dev_f2_squad_2.describe())

df_train_f1_squad_2.to_parquet('../data/df_train_tdms_long_summarized_f1_squad_2.parquet')
df_dev_f1_squad_2.to_parquet('../data/df_dev_tdms_long_summarized_f1_squad_2.parquet')
df_train_f2_squad_2.to_parquet('../data/df_train_tdms_long_summarized_f2_squad_2.parquet')
df_dev_f2_squad_2.to_parquet('../data/df_dev_tdms_long_summarized_f2_squad_2.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_squad_2.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_squad_2.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_squad_2.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_squad_2.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_SQUAD_2")

  1%|▏         | 74/5512 [00:00<00:07, 734.60it/s]

100%|██████████| 2352/2352 [00:02<00:00, 842.98it/s]

df_train_f1_squad_2 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,Read this and answer the question. If the ques...,unanswerable
freq,2,1872


df_dev_f1_squad_2 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-ea3cf5ca9e08c92f/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [26]:
df_train_f1_squad_3 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        squad_3 = True,
                                                        ) 
df_dev_f1_squad_3 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        squad_3 = True,
                                                        ) 
df_train_f2_squad_3 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        squad_3 = True,
                                                        ) 
df_dev_f2_squad_3 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        squad_3 = True,
                                                        ) 

print("df_train_f1_squad_3 describe: ")
display(df_train_f1_squad_3.describe())
print("df_dev_f1_squad_3 describe: ")
display(df_dev_f1_squad_3.describe())

print("df_train_f2_squad_3 describe: ")
display(df_train_f2_squad_3.describe())
print("df_dev_f2_squad_3 describe: ")
display(df_dev_f2_squad_3.describe())

df_train_f1_squad_3.to_parquet('../data/df_train_tdms_long_summarized_f1_squad_3.parquet')
df_dev_f1_squad_3.to_parquet('../data/df_dev_tdms_long_summarized_f1_squad_3.parquet')
df_train_f2_squad_3.to_parquet('../data/df_train_tdms_long_summarized_f2_squad_3.parquet')
df_dev_f2_squad_3.to_parquet('../data/df_dev_tdms_long_summarized_f2_squad_3.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_squad_3.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_squad_3.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_squad_3.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_squad_3.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_SQUAD_3")

  1%|▏         | 72/5512 [00:00<00:07, 713.72it/s]

100%|██████████| 2352/2352 [00:02<00:00, 848.33it/s]

df_train_f1_squad_3 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable
freq,2,1872


df_dev_f1_squad_3 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-d0b3fea1c6de045c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [27]:
df_train_f1_squad_4 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        squad_4 = True,
                                                        ) 
df_dev_f1_squad_4 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        squad_4 = True,
                                                        ) 
df_train_f2_squad_4 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        squad_4 = True,
                                                        ) 
df_dev_f2_squad_4 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        squad_4 = True,
                                                        ) 

print("df_train_f1_squad_4 describe: ")
display(df_train_f1_squad_4.describe())
print("df_dev_f1_squad_4 describe: ")
display(df_dev_f1_squad_4.describe())

print("df_train_f2_squad_4 describe: ")
display(df_train_f2_squad_4.describe())
print("df_dev_f2_squad_4 describe: ")
display(df_dev_f2_squad_4.describe())

df_train_f1_squad_4.to_parquet('../data/df_train_tdms_long_summarized_f1_squad_4.parquet')
df_dev_f1_squad_4.to_parquet('../data/df_dev_tdms_long_summarized_f1_squad_4.parquet')
df_train_f2_squad_4.to_parquet('../data/df_train_tdms_long_summarized_f2_squad_4.parquet')
df_dev_f2_squad_4.to_parquet('../data/df_dev_tdms_long_summarized_f2_squad_4.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_squad_4.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_squad_4.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_squad_4.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_squad_4.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_SQUAD_4")

  1%|▏         | 69/5512 [00:00<00:07, 686.16it/s]

100%|██████████| 2352/2352 [00:02<00:00, 846.04it/s]

df_train_f1_squad_4 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable
freq,2,1872


df_dev_f1_squad_4 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-e5a405cc1e529695/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [28]:
df_train_f1_squad_5 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        squad_5 = True,
                                                        ) 
df_dev_f1_squad_5 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        squad_5 = True,
                                                        ) 
df_train_f2_squad_5 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        squad_5 = True,
                                                        ) 
df_dev_f2_squad_5 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        squad_5 = True,
                                                        ) 

print("df_train_f1_squad_5 describe: ")
display(df_train_f1_squad_5.describe())
print("df_dev_f1_squad_5 describe: ")
display(df_dev_f1_squad_5.describe())

print("df_train_f2_squad_5 describe: ")
display(df_train_f2_squad_5.describe())
print("df_dev_f2_squad_5 describe: ")
display(df_dev_f2_squad_5.describe())

df_train_f1_squad_5.to_parquet('../data/df_train_tdms_long_summarized_f1_squad_5.parquet')
df_dev_f1_squad_5.to_parquet('../data/df_dev_tdms_long_summarized_f1_squad_5.parquet')
df_train_f2_squad_5.to_parquet('../data/df_train_tdms_long_summarized_f2_squad_5.parquet')
df_dev_f2_squad_5.to_parquet('../data/df_dev_tdms_long_summarized_f2_squad_5.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_squad_5.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_squad_5.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_squad_5.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_squad_5.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_SQUAD_5")

  1%|▏         | 74/5512 [00:00<00:07, 733.65it/s]

100%|██████████| 2352/2352 [00:02<00:00, 839.37it/s]

df_train_f1_squad_5 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable
freq,2,1872


df_dev_f1_squad_5 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-82953ddb8286bb12/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [29]:
df_train_f1_squad_6 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        squad_6 = True,
                                                        ) 
df_dev_f1_squad_6 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        squad_6 = True,
                                                        ) 
df_train_f2_squad_6 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        squad_6 = True,
                                                        ) 
df_dev_f2_squad_6 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        squad_6 = True,
                                                        ) 

print("df_train_f1_squad_6 describe: ")
display(df_train_f1_squad_6.describe())
print("df_dev_f1_squad_6 describe: ")
display(df_dev_f1_squad_6.describe())

print("df_train_f2_squad_6 describe: ")
display(df_train_f2_squad_6.describe())
print("df_dev_f2_squad_6 describe: ")
display(df_dev_f2_squad_6.describe())

df_train_f1_squad_6.to_parquet('../data/df_train_tdms_long_summarized_f1_squad_6.parquet')
df_dev_f1_squad_6.to_parquet('../data/df_dev_tdms_long_summarized_f1_squad_6.parquet')
df_train_f2_squad_6.to_parquet('../data/df_train_tdms_long_summarized_f2_squad_6.parquet')
df_dev_f2_squad_6.to_parquet('../data/df_dev_tdms_long_summarized_f2_squad_6.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_squad_6.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_squad_6.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_squad_6.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_squad_6.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_SQUAD_6")

  1%|▏         | 72/5512 [00:00<00:07, 717.91it/s]

100%|██████████| 2352/2352 [00:02<00:00, 832.25it/s]


df_train_f1_squad_6 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable
freq,2,1872


df_dev_f1_squad_6 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-4ab7d626f2bdad62/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [30]:
df_train_f1_squad_7 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        squad_7 = True,
                                                        ) 
df_dev_f1_squad_7 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        squad_7 = True,
                                                        ) 
df_train_f2_squad_7 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        squad_7 = True,
                                                        ) 
df_dev_f2_squad_7 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        squad_7 = True,
                                                        ) 

print("df_train_f1_squad_7 describe: ")
display(df_train_f1_squad_7.describe())
print("df_dev_f1_squad_7 describe: ")
display(df_dev_f1_squad_7.describe())

print("df_train_f2_squad_7 describe: ")
display(df_train_f2_squad_7.describe())
print("df_dev_f2_squad_7 describe: ")
display(df_dev_f2_squad_7.describe())

df_train_f1_squad_7.to_parquet('../data/df_train_tdms_long_summarized_f1_squad_7.parquet')
df_dev_f1_squad_7.to_parquet('../data/df_dev_tdms_long_summarized_f1_squad_7.parquet')
df_train_f2_squad_7.to_parquet('../data/df_train_tdms_long_summarized_f2_squad_7.parquet')
df_dev_f2_squad_7.to_parquet('../data/df_dev_tdms_long_summarized_f2_squad_7.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_squad_7.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_squad_7.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_squad_7.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_squad_7.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_SQUAD_7")

 14%|█▎        | 751/5512 [00:00<00:05, 833.27it/s]

100%|██████████| 2352/2352 [00:02<00:00, 838.84it/s]

df_train_f1_squad_7 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,Read this: FixMatch: Simplifying Semi-Supervis...,unanswerable
freq,2,1872


df_dev_f1_squad_7 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-416c92763140e571/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [31]:
df_train_f1_squad_8 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        squad_8 = True,
                                                        ) 
df_dev_f1_squad_8 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        squad_8 = True,
                                                        ) 
df_train_f2_squad_8 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        squad_8 = True,
                                                        ) 
df_dev_f2_squad_8 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        squad_8 = True,
                                                        ) 

print("df_train_f1_squad_8 describe: ")
display(df_train_f1_squad_8.describe())
print("df_dev_f1_squad_8 describe: ")
display(df_dev_f1_squad_8.describe())

print("df_train_f2_squad_8 describe: ")
display(df_train_f2_squad_8.describe())
print("df_dev_f2_squad_8 describe: ")
display(df_dev_f2_squad_8.describe())

df_train_f1_squad_8.to_parquet('../data/df_train_tdms_long_summarized_f1_squad_8.parquet')
df_dev_f1_squad_8.to_parquet('../data/df_dev_tdms_long_summarized_f1_squad_8.parquet')
df_train_f2_squad_8.to_parquet('../data/df_train_tdms_long_summarized_f2_squad_8.parquet')
df_dev_f2_squad_8.to_parquet('../data/df_dev_tdms_long_summarized_f2_squad_8.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_squad_8.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_squad_8.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_squad_8.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_squad_8.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_SQUAD_8")

  1%|▏         | 76/5512 [00:00<00:07, 753.38it/s]

100%|██████████| 2352/2352 [00:02<00:00, 834.24it/s]


df_train_f1_squad_8 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,Read this: FixMatch: Simplifying Semi-Supervis...,unanswerable
freq,2,1872


df_dev_f1_squad_8 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-0c8226ba5e124291/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [32]:
df_train_f1_drop_1 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        drop_1 = True,
                                                        ) 
df_dev_f1_drop_1 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        drop_1 = True,
                                                        ) 
df_train_f2_drop_1 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        drop_1 = True,
                                                        ) 
df_dev_f2_drop_1 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        drop_1 = True,
                                                        ) 

print("df_train_f1_drop_1 describe: ")
display(df_train_f1_drop_1.describe())
print("df_dev_f1_drop_1 describe: ")
display(df_dev_f1_drop_1.describe())

print("df_train_f2_drop_1 describe: ")
display(df_train_f2_drop_1.describe())
print("df_dev_f2_drop_1 describe: ")
display(df_dev_f2_drop_1.describe())

df_train_f1_drop_1.to_parquet('../data/df_train_tdms_long_summarized_f1_drop_1.parquet')
df_dev_f1_drop_1.to_parquet('../data/df_dev_tdms_long_summarized_f1_drop_1.parquet')
df_train_f2_drop_1.to_parquet('../data/df_train_tdms_long_summarized_f2_drop_1.parquet')
df_dev_f2_drop_1.to_parquet('../data/df_dev_tdms_long_summarized_f2_drop_1.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_drop_1.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_drop_1.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_drop_1.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_drop_1.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_DROP_1")

100%|██████████| 2352/2352 [00:02<00:00, 814.21it/s]

df_train_f1_drop_1 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,Answer based on context:\n\nFixMatch: Simplify...,unanswerable
freq,2,1872


df_dev_f1_drop_1 describe: 


,prompt,answer
count,2353,2353
unique,2352,1549
top,Answer based on context:\n\nPANDA: Adapting Pr...,unanswerable
freq,2,804


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-e3078e6d00ec8ffe/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [33]:
df_train_f1_drop_2 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        drop_2 = True,
                                                        ) 
df_dev_f1_drop_2 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        drop_2 = True,
                                                        ) 
df_train_f2_drop_2 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        drop_2 = True,
                                                        ) 
df_dev_f2_drop_2 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        drop_2 = True,
                                                        ) 

print("df_train_f1_drop_2 describe: ")
display(df_train_f1_drop_2.describe())
print("df_dev_f1_drop_2 describe: ")
display(df_dev_f1_drop_2.describe())

print("df_train_f2_drop_2 describe: ")
display(df_train_f2_drop_2.describe())
print("df_dev_f2_drop_2 describe: ")
display(df_dev_f2_drop_2.describe())

df_train_f1_drop_2.to_parquet('../data/df_train_tdms_long_summarized_f1_drop_2.parquet')
df_dev_f1_drop_2.to_parquet('../data/df_dev_tdms_long_summarized_f1_drop_2.parquet')
df_train_f2_drop_2.to_parquet('../data/df_train_tdms_long_summarized_f2_drop_2.parquet')
df_dev_f2_drop_2.to_parquet('../data/df_dev_tdms_long_summarized_f2_drop_2.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_drop_2.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_drop_2.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_drop_2.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_drop_2.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_DROP_2")

  7%|▋         | 398/5512 [00:00<00:06, 802.94it/s]

100%|██████████| 2352/2352 [00:02<00:00, 849.43it/s]

df_train_f1_drop_2 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable
freq,2,1872


df_dev_f1_drop_2 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-88783fd410f1e582/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [34]:
df_train_f1_drop_3 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        drop_3 = True,
                                                        ) 
df_dev_f1_drop_3 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        drop_3 = True,
                                                        ) 
df_train_f2_drop_3 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        drop_3 = True,
                                                        ) 
df_dev_f2_drop_3 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        drop_3 = True,
                                                        ) 

print("df_train_f1_drop_3 describe: ")
display(df_train_f1_drop_3.describe())
print("df_dev_f1_drop_3 describe: ")
display(df_dev_f1_drop_3.describe())

print("df_train_f2_drop_3 describe: ")
display(df_train_f2_drop_3.describe())
print("df_dev_f2_drop_3 describe: ")
display(df_dev_f2_drop_3.describe())

df_train_f1_drop_3.to_parquet('../data/df_train_tdms_long_summarized_f1_drop_3.parquet')
df_dev_f1_drop_3.to_parquet('../data/df_dev_tdms_long_summarized_f1_drop_3.parquet')
df_train_f2_drop_3.to_parquet('../data/df_train_tdms_long_summarized_f2_drop_3.parquet')
df_dev_f2_drop_3.to_parquet('../data/df_dev_tdms_long_summarized_f2_drop_3.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_drop_3.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_drop_3.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_drop_3.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_drop_3.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_DROP_3")

  1%|▏         | 75/5512 [00:00<00:07, 746.80it/s]

100%|██████████| 2352/2352 [00:02<00:00, 838.48it/s]

df_train_f1_drop_3 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable
freq,2,1872


df_dev_f1_drop_3 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-d74b23353d41d39a/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [35]:
df_train_f1_drop_4 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        drop_4 = True,
                                                        ) 
df_dev_f1_drop_4 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        drop_4 = True,
                                                        ) 
df_train_f2_drop_4 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        drop_4 = True,
                                                        ) 
df_dev_f2_drop_4 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        drop_4 = True,
                                                        ) 

print("df_train_f1_drop_4 describe: ")
display(df_train_f1_drop_4.describe())
print("df_dev_f1_drop_4 describe: ")
display(df_dev_f1_drop_4.describe())

print("df_train_f2_drop_4 describe: ")
display(df_train_f2_drop_4.describe())
print("df_dev_f2_drop_4 describe: ")
display(df_dev_f2_drop_4.describe())

df_train_f1_drop_4.to_parquet('../data/df_train_tdms_long_summarized_f1_drop_4.parquet')
df_dev_f1_drop_4.to_parquet('../data/df_dev_tdms_long_summarized_f1_drop_4.parquet')
df_train_f2_drop_4.to_parquet('../data/df_train_tdms_long_summarized_f2_drop_4.parquet')
df_dev_f2_drop_4.to_parquet('../data/df_dev_tdms_long_summarized_f2_drop_4.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_drop_4.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_drop_4.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_drop_4.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_drop_4.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_DROP_4")

  3%|▎         | 163/5512 [00:00<00:06, 819.60it/s]

100%|██████████| 2352/2352 [00:02<00:00, 828.51it/s]

df_train_f1_drop_4 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable
freq,2,1872


df_dev_f1_drop_4 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-d8307e5e113cfffb/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [36]:
df_train_f1_drop_5 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        drop_5 = True,
                                                        ) 
df_dev_f1_drop_5 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        drop_5 = True,
                                                        ) 
df_train_f2_drop_5 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        drop_5 = True,
                                                        ) 
df_dev_f2_drop_5 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        drop_5 = True,
                                                        ) 

print("df_train_f1_drop_5 describe: ")
display(df_train_f1_drop_5.describe())
print("df_dev_f1_drop_5 describe: ")
display(df_dev_f1_drop_5.describe())

print("df_train_f2_drop_5 describe: ")
display(df_train_f2_drop_5.describe())
print("df_dev_f2_drop_5 describe: ")
display(df_dev_f2_drop_5.describe())

df_train_f1_drop_5.to_parquet('../data/df_train_tdms_long_summarized_f1_drop_5.parquet')
df_dev_f1_drop_5.to_parquet('../data/df_dev_tdms_long_summarized_f1_drop_5.parquet')
df_train_f2_drop_5.to_parquet('../data/df_train_tdms_long_summarized_f2_drop_5.parquet')
df_dev_f2_drop_5.to_parquet('../data/df_dev_tdms_long_summarized_f2_drop_5.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_drop_5.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_drop_5.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_drop_5.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_drop_5.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_DROP_5")

  3%|▎         | 172/5512 [00:00<00:06, 856.01it/s]

100%|██████████| 2352/2352 [00:02<00:00, 836.79it/s]


df_train_f1_drop_5 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,Read this article and answer this question Fix...,unanswerable
freq,2,1872


df_dev_f1_drop_5 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-905a4a0dc714d555/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [37]:
df_train_f1_drop_6 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        drop_6 = True,
                                                        ) 
df_dev_f1_drop_6 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        drop_6 = True,
                                                        ) 
df_train_f2_drop_6 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        drop_6 = True,
                                                        ) 
df_dev_f2_drop_6 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        drop_6 = True,
                                                        ) 

print("df_train_f1_drop_6 describe: ")
display(df_train_f1_drop_6.describe())
print("df_dev_f1_drop_6 describe: ")
display(df_dev_f1_drop_6.describe())

print("df_train_f2_drop_6 describe: ")
display(df_train_f2_drop_6.describe())
print("df_dev_f2_drop_6 describe: ")
display(df_dev_f2_drop_6.describe())

df_train_f1_drop_6.to_parquet('../data/df_train_tdms_long_summarized_f1_drop_6.parquet')
df_dev_f1_drop_6.to_parquet('../data/df_dev_tdms_long_summarized_f1_drop_6.parquet')
df_train_f2_drop_6.to_parquet('../data/df_train_tdms_long_summarized_f2_drop_6.parquet')
df_dev_f2_drop_6.to_parquet('../data/df_dev_tdms_long_summarized_f2_drop_6.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_drop_6.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_drop_6.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_drop_6.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_drop_6.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_DROP_6")

  1%|▏         | 73/5512 [00:00<00:07, 723.47it/s]

100%|██████████| 2352/2352 [00:02<00:00, 841.62it/s]

df_train_f1_drop_6 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable
freq,2,1872


df_dev_f1_drop_6 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-f1d8e02f76e5aba5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [38]:
df_train_f1_drop_7 = create_pandas_dataset_from_pandas(df_train_f1,
                                                        drop_7 = True,
                                                        ) 
df_dev_f1_drop_7 = create_pandas_dataset_from_pandas(df_dev_f1,
                                                        drop_7 = True,
                                                        ) 
df_train_f2_drop_7 = create_pandas_dataset_from_pandas(df_train_f2,
                                                        drop_7 = True,
                                                        ) 
df_dev_f2_drop_7 = create_pandas_dataset_from_pandas(df_dev_f2,
                                                        drop_7 = True,
                                                        ) 

print("df_train_f1_drop_7 describe: ")
display(df_train_f1_drop_7.describe())
print("df_dev_f1_drop_7 describe: ")
display(df_dev_f1_drop_7.describe())

print("df_train_f2_drop_7 describe: ")
display(df_train_f2_drop_7.describe())
print("df_dev_f2_drop_7 describe: ")
display(df_dev_f2_drop_7.describe())

df_train_f1_drop_7.to_parquet('../data/df_train_tdms_long_summarized_f1_drop_7.parquet')
df_dev_f1_drop_7.to_parquet('../data/df_dev_tdms_long_summarized_f1_drop_7.parquet')
df_train_f2_drop_7.to_parquet('../data/df_train_tdms_long_summarized_f2_drop_7.parquet')
df_dev_f2_drop_7.to_parquet('../data/df_dev_tdms_long_summarized_f2_drop_7.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_drop_7.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_drop_7.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_drop_7.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_drop_7.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_DROP_7")

  1%|▏         | 72/5512 [00:00<00:07, 716.64it/s]

100%|██████████| 2352/2352 [00:02<00:00, 829.43it/s]


df_train_f1_drop_7 describe: 


,prompt,answer
count,5512,5512
unique,5510,3634
top,Context: FixMatch: Simplifying Semi-Supervised...,unanswerable
freq,2,1872


df_dev_f1_drop_7 describe: 


...

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-b2d7f05fd261d427/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5512
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2353
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 5513
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 2352
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/5512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2353 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2352 [00:00<?, ? examples/s]

In [44]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [39]:
# root_directory = "../data/LLLM_LONG_SUMMARIZED_TDMS_ALL_TEMPLATE"
root_directory = "../data/LLLM_LONG_SUMMARIZED_TDMS_DROP_7"


# reloaded_encoded_dataset = datasets.load_from_disk("../data/dataset/LLLM_TDMS_ALL_TEMPLATE")
# reloaded_encoded_dataset = DatasetDict.load_from_disk("../data/LLLM_TDMS_ALL_TEMPLATE")

dataset_fold1 = DatasetDict.load_from_disk(f"{root_directory}/fold1")
dataset_fold2 = DatasetDict.load_from_disk(f"{root_directory}/fold2")

In [40]:
dataset_fold1['train'][0]

{'prompt': "Context: Value Prediction Network This paper proposes a novel deep reinforcement learning (RL) architecture, called Value Prediction Network (VPN), which integrates model-free and model-based RL methods into a single neural network. In contrast to typical model-based RL methods, VPN learns a dynamics model whose abstract states are trained to make option-conditional predictions of future values (discounted sum of rewards) rather than of future observations. Our experimental results show that VPN has several advantages over both model-free and model-based baselines in a stochastic environment where careful planning is required but building an accurate observation-prediction model is difficult. Furthermore, VPN outperforms Deep Q-Network (DQN) on several Atari games even with short-lookahead planning, demonstrating its potential as anew way of learning a good state representation. VPN has four more hyperparameters: 1) the number of predictions steps (k) during training, 2) th

In [21]:
template = ['Please answer a question about this article. If the question is unanswerable, say \"unanswerable\"',
'Read this and answer the question. If the question is unanswerable, say \"unanswerable\".',
'If the question is unanswarable, say \"unanswerable\"',
'Try to answer this question if possible (otherwise reply \"unanswerable\"',
'If it is possible to answer this question, answer it for me (else, reply \"unanswerable\"',
'Answer this question, if possible (if impossible, reply \"unanswerable\"',
'Read this: What is the answer? (If it cannot be answered, return \"unanswerable\"',
'Read this: Now answer this question, if there is an answer (If it cannot be answered, return \"unanswerable\"',
'Answer based on context:',
'Answer this question based on theaarticle:',
# ("{context}\n\n{question}", "{answer}"),
'Answer this question:',
'Read this article and answer this question',
'Based on the above article, answer a question.',
# 'Context: {context}\n\nQuestion: {question}\n\nAnswer:", "{answer}"),
]

template

['Please answer a question about this article. If the question is unanswerable, say "unanswerable"',
 'Read this and answer the question. If the question is unanswerable, say "unanswerable".',
 'If the question is unanswerable, say "unanswerable"',
 'Try to answer this question if possible (otherwise reply "unanswerable"',
 'If it is possible to answer this question, answer it for me (else, reply "unanswerable"',
 'Answer this question, if possible (if impossible, reply "unanswerable"',
 'Read this: What is the answer? (If it cannot be answered, return "unanswerable"',
 'Read this: Now answer this question, if there is an answer (If it cannot be answered, return "unanswerable"',
 'Answer based on context:',
 'Answer this question based on the article:',
 'Answer this question:',
 'Read this article and answer this question',
 'Based on the above article, answer a question.']

In [22]:
# df.head(2)

In [23]:
def create_pandas_dataset_from_pandas(df,
                          answer_threshold=7,
                          verbose = False):

  ''' Create a Pandas Dataframe from pandas.
  Params:
        answer_threshold: Only consider those Question Answer pairs where the Answer is short.
  '''
  count_index = 0
  result_df  = pd.DataFrame(columns = ['prompt', 'answer'])   
  # q_type_1 = "Which Tasks are addressed in this article"
  # q_type_2 = "Which Datasets are addressed in this article"
  # q_type_3 = "Which Metrics are addressed in this article"
  # q_type_4 = "Which Tasks, Datasets, Metrics are addressed in this article"
  # q_type_5 = "Which Tasks, Datasets, Metrics and Scores are addressed in this article" 
  
  # q_types = [
  #   {"q": "Which Tasks are addressed in this article", "a_key": "Tasks"}, 
  #   {"q": "Which Datasets are addressed in this article", "a_key": "Datasets"}, 
  #   {"q": "Which Metrics are addressed in this article", "a_key": "Metrics"},
  #   {"q": "Which Tasks, Datasets, Metrics are addressed in this article", "a_key": "TDMs"},
  #   {"q": "Which Tasks, Datasets, Metrics and Scores are addressed in this article", "a_key": "TDMSs"}
  #   ]
  
  q_types = [
    # {"q": "What are the values for the following properties to construct a Leaderboard for the model introduced in this article: task, dataset, and metric?", "a_key": "TDMSs"},
    {"q": "What are the values for the following properties to construct a Leaderboard for the model introduced in this article: task, dataset, metric, and score?", "a_key": "TDMSs"},
    ]
  
  records = df.to_dict("records")
  # db_dict = defaultdict(lambda : list())
  for i, row in tqdm(enumerate(records), total = len(records)):        
      for q_type in q_types:
        
        # Squad_v2
        result_df.loc[count_index] = [f'{row["Context"]}\n\nPlease answer a question about this article. If the question is unanswerable, say \"unanswerable\". {q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'Read this and answer the question. If the question is unanswerable, say \"unanswerable\".\n\n{row["Context"]}\n\n{q_type["q"]}'
] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'{row["Context"]}\n{q_type["q"]} (If the question is unanswerable, say \"unanswerable\"'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'{row["Context"]}\nTry to answer this question if possible (otherwise reply \"unanswerable\"): {q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'{row["Context"]}\nIf it is possible to answer this question, answer it for me (else, reply \"unanswerable\"): {q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'{row["Context"]}\n\nAnswer this question, if possible (if impossible, reply \"unanswerable\"): {q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'Read this: {row["Context"]}\n\n{q_type["q"]}\nWhat is the answer? (If it cannot be answered, return \"unanswerable\")'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'Read this: {row["Context"]}\nNow answer this question, if there is an answer (If it cannot be answered, return \"unanswerable\"): {q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        
        # Drop
        result_df.loc[count_index] = [f'Answer based on context:\n\n{row["Context"]}\n\n{q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'{row["Context"]}\n\nAnswer this question based on the article: {q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'{row["Context"]}\n\n{q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'{row["Context"]}\nAnswer this question: {q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'Read this article and answer this question {row["Context"]}\n{q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'{row["Context"]}\n\nBased on the above article, answer a question. {q_type["q"]}'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
        result_df.loc[count_index] = [f'Context: {row["Context"]}\n\nQuestion: {q_type["q"]}\n\nAnswer:'] \
          + [str(row[q_type["a_key"]])] 
        count_index += 1
         
            
  if verbose:
    # return (result_df,
    #         count_long,
    #         count_short)
    return (result_df)
  else:
    return result_df

In [24]:
df_train_f1_all_templates = create_pandas_dataset_from_pandas(df_train_f1) 
df_dev_f1_all_templates = create_pandas_dataset_from_pandas(df_dev_f1) 
df_train_f2_all_templates = create_pandas_dataset_from_pandas(df_train_f2) 
df_dev_f2_all_templates = create_pandas_dataset_from_pandas(df_dev_f2) 

  0%|          | 4/5512 [00:00<03:02, 30.26it/s]

100%|██████████| 2352/2352 [01:19<00:00, 29.60it/s]


In [25]:
df_train_f1_all_templates.describe()

,prompt,answer
count,82680,82680
unique,82335,3634
top,Title\tLaTeX Author Guidelines for CVPR Procee...,unanswerable
freq,5,28080


In [26]:
print("df_train_f1_all_templates describe: ")
display(df_train_f1_all_templates.describe())
print("df_dev_f1_all_templates describe: ")
display(df_dev_f1_all_templates.describe())

print("df_train_f2_all_templates describe: ")
display(df_train_f2_all_templates.describe())
print("df_dev_f2_all_templates describe: ")
display(df_dev_f2_all_templates.describe())

df_train_f1_all_templates describe: 


,prompt,answer
count,82680,82680
unique,82335,3634
top,Title\tLaTeX Author Guidelines for CVPR Procee...,unanswerable
freq,5,28080


df_dev_f1_all_templates describe: 


,prompt,answer
count,35295,35295
unique,35145,1549
top,Read this: Title\tLaTeX Author Guidelines for ...,unanswerable
freq,5,12060


df_train_f2_all_templates describe: 


,prompt,answer
count,82695,82695
unique,82260,3637
top,Title\tLaTeX Author Guidelines for CVPR Procee...,unanswerable
freq,6,28080


df_dev_f2_all_templates describe: 


,prompt,answer
count,35280,35280
unique,35205,1548
top,Title\tBare Advanced Demo of IEEEtran.cls for\...,unanswerable
freq,2,12060


In [27]:
df_train_f1_all_templates['answer'].apply(type).value_counts()


<class 'str'>    82680
Name: answer, dtype: int64

In [28]:
df_dev_f1_all_templates['answer'].apply(type).value_counts()


<class 'str'>    35295
Name: answer, dtype: int64

In [29]:
# df_train_f1_all_templates.to_parquet('../data/df_train_long_f1_all_templates.parquet')
# df_dev_f1_all_templates.to_parquet('../data/df_dev_long_f1_all_templates.parquet')
# df_train_f2_all_templates.to_parquet('../data/df_train_long_f2_all_templates.parquet')
# df_dev_f2_all_templates.to_parquet('../data/df_dev_long_f2_all_templates.parquet')

df_train_f1_all_templates.to_parquet('../data/df_train_tdms_long_summarized_f1_all_templates.parquet')
df_dev_f1_all_templates.to_parquet('../data/df_dev_tdms_long_summarized_f1_all_templates.parquet')
df_train_f2_all_templates.to_parquet('../data/df_train_tdms_long_summarized_f2_all_templates.parquet')
df_dev_f2_all_templates.to_parquet('../data/df_dev_tdms_long_summarized_f2_all_templates.parquet')

In [30]:
# df_train_f1_all_templates = pd.read_parquet('../data/df_train_f1_all_templates.parquet')
# df_dev_f1_all_templates = pd.read_parquet('../data/df_dev_f1_all_templates.parquet')
# df_train_f2_all_templates = pd.read_parquet('../data/df_train_f2_all_templates.parquet')
# df_dev_f2_all_templates = pd.read_parquet('../data/df_dev_f2_all_templates.parquet')

In [31]:
# dataset_train_f1_all_templates = Dataset.from_pandas(df_train_f1_all_templates)
# dataset_dev_f1_all_templates = Dataset.from_pandas(df_dev_f1_all_templates)
# dataset_train_f2_all_templates = Dataset.from_pandas(df_train_f2_all_templates)
# dataset_dev_f2_all_templates = Dataset.from_pandas(df_dev_f2_all_templates)



# f1 = DatasetDict({
#         "train": dataset_train_f1_all_templates,
#         "validation": dataset_dev_f1_all_templates
#     })
# f2 = DatasetDict({
#         "train": dataset_train_f2_all_templates,
#         "validation": dataset_dev_f2_all_templates
#     })

# Combine into a DatasetDict
# dataset = DatasetDict({
#     'fold1': DatasetDict({
#         "train": dataset_train_f1_all_templates,
#         "validation": dataset_dev_f1_all_templates
#     }),
#     'fold2': DatasetDict({
#         "train": dataset_train_f2_all_templates,
#         "validation": dataset_dev_f2_all_templates
#     })
# })


dataset = DatasetDict({
    'fold1': DatasetDict({
        # "train": Dataset.from_parquet('../data/df_train_long_f1_all_templates.parquet'),
        # "validation": Dataset.from_parquet('../data/df_dev_long_f1_all_templates.parquet'),
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f1_all_templates.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f1_all_templates.parquet'),
    }),
    'fold2': DatasetDict({
        # "train": Dataset.from_parquet('../data/df_train_long_f2_all_templates.parquet'),
        # "validation": Dataset.from_parquet('../data/df_dev_long_f2_all_templates.parquet'),
        "train": Dataset.from_parquet('../data/df_train_tdms_long_summarized_f2_all_templates.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_summarized_f2_all_templates.parquet'),
    })
})

# dataset_fold1 = DatasetDict({
#     "train": Dataset.from_parquet('../data/df_train_f1_all_templates.parquet'),
#     "validation": Dataset.from_parquet('../data/df_dev_f1_all_templates.parquet')
#     })

# dataset_fold2 =  DatasetDict({
#     "train": Dataset.from_parquet('../data/df_train_f2_all_templates.parquet'),
#     "validation": Dataset.from_parquet('../data/df_dev_f2_all_templates.parquet')
#     })

# dataset = DatasetDict({
#     'fold1': f1,
#     'fold2': f2
# })

# dataset = DatasetDict({
#     "train": dataset_train_f1_all_templates,
#     "validation": dataset_dev_f1_all_templates
#     })

print(dataset)
# print(dataset_fold1)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-ac107c7899e6dda8/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-cab000074a3fedd8/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-cd61d162599e5da7/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 82680
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 35295
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 82695
        })
        validation: Dataset({
            features: ['prompt', 'answer', '__index_level_0__'],
            num_rows: 35280
        })
    })
})


In [32]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [33]:
dataset.save_to_disk("../data/LLLM_LONG_SUMMARIZED_TDMS_ALL_TEMPLATE")

Saving the dataset (0/5 shards):   0%|          | 0/82680 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/35295 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/82695 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/35280 [00:00<?, ? examples/s]

In [4]:
# root_directory = "../data/LLLM_LONG_TDM_ALL_TEMPLATE"
# root_directory = "../data/LLLM_LONG_SUMMARIZED_TDMS_ALL_TEMPLATE"
# root_directory = "../data/LLLM_LONG_TDM_ALL_TEMPLATE"
root_directory = "../data/LLLM_LONG_SUMMARIZED_TDMS_ALL_TEMPLATE"

# reloaded_encoded_dataset = datasets.load_from_disk("../data/dataset/LLLM_TDMS_ALL_TEMPLATE")
# reloaded_encoded_dataset = DatasetDict.load_from_disk("../data/LLLM_TDMS_ALL_TEMPLATE")

dataset_fold1 = DatasetDict.load_from_disk(f"{root_directory}/fold1")
dataset_fold2 = DatasetDict.load_from_disk(f"{root_directory}/fold2")

In [10]:
dataset_fold1['train'][1900]

{'prompt': 'Local Class-Specific and Global Image-Level Generative Adversarial Networks for Semantic-Guided Scene Generation In this paper, we address the task of semantic-guided scene generation. One open challenge widely observed in global image-level generation methods is the difficulty of generating small objects and detailed local texture. To tackle this issue, in this work we consider learning the scene generation in a local context, and correspondingly design a local class-specific generative network with semantic maps as a guidance, which separately constructs and learns sub-generators concentrating on the generation of different classes, and is able to provide more scene details. To learn more discriminative class-specific feature representations for the local generation, a novel classification module is also proposed. To combine the advantage of both global image-level and the local class-specific generation, a joint generation network is designed with an attention fusion mod

In [6]:
len(dataset_fold1['train'])

82680

In [ ]:
arxiv_leaderboard_full_txt = "\
/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_txt_summarised_dec092023"

print(f"Total available papers: {len(os.listdir(arxiv_leaderboard_full_txt))}\n\n")

print(f'Total train f1 before adding {len(title_to_content["train_f1"].keys())}')
print(f'Total dev f1 {len(title_to_content["dev_f1"].keys())}')

missed = 0

for file_id_sum_txt in os.listdir(arxiv_leaderboard_full_txt):
    
    file_id = file_id_sum_txt.rsplit("_", 1)[0]

    if f"{file_id}.pdf" in title_to_content["dev_f1"]:
        # ipdb.set_trace()
        continue 
    
    try:
        with open(f'{arxiv_leaderboard_full_txt}/{file_id}_summarised.txt', 'r') as file:
            # Read the file
            data = file.read()

        # ipdb.set_trace()
        
        title_to_content["train_f1"][f"{file_id}.pdf"] = data
        
    except :
        # print(f"Error on file {row['title']}")
        missed += 1
        ipdb.set_trace()
        # continue

print(f"Missed reading F1 {missed}")
print(f'Total train f1 after adding {len(title_to_content["train_f1"].keys())}\n')


missed = 0
print(f'Total train f2 before adding {len(title_to_content["train_f2"].keys())}')
print(f'Total dev f2 {len(title_to_content["dev_f2"].keys())}')

for file_id_sum_txt in os.listdir(arxiv_leaderboard_full_txt):
    
    file_id = file_id_sum_txt.rsplit("_", 1)[0]

    if f"{file_id}.pdf" in title_to_content["dev_f2"]:
        # ipdb.set_trace()
        continue 
    
    try:
        with open(f'{arxiv_leaderboard_full_txt}/{file_id}_summarised.txt', 'r') as file:
            # Read the file
            data = file.read()

        # ipdb.set_trace()
        
        title_to_content["train_f2"][f"{file_id}.pdf"] = data
        
    except :
        # print(f"Error on file {row['title']}")
        missed += 1
        ipdb.set_trace()
        # continue

print(f"Missed reading F2 {missed}")
print(f'Total train f2 after adding {len(title_to_content["train_f2"].keys())}')

In [ ]:
missed_parsing = 0
missed_short_context = 0
copied = 0
for i, row in tqdm(enumerate(records_dev_f2), total = len(records_dev_f2)):
    title_id = row['title'].split(".pdf")[0]
    if row['title'] in title_to_content["dev_f2"]:
        continue 
    else:
        try:
            with open(f'{arxiv_leaderboard_full_txt}/{title_id}_summarised.txt', 'r') as file:
                # Read the file
                data = file.read()
                
        except :
            # print(f"Error on file {row['title']}")
            missed_parsing += 1
            continue 
            
        # title_to_content["dev_f2"][row['title']] = row['Context'] if len(data.split()) < 100 else data
        # title_to_content["dev_f2"][row['title']] = row['Context']
        if len(data.split()) < 10:
            missed_short_context += 1
            continue
        else:
            title_to_content["dev_f2"][row['title']] = data
            copied += 1
            
print(f"DEV F2:\nCopied: {copied}\nMissed Parsing: {missed_parsing}\
\nMissed Short Context: {missed_short_context}\nAll items: {len(records_dev_f2)}\n") 